# Coastal Erosion Hazard
This notebook calculates the urban risk index for Coastal Erosion (CER)
hazard.  The URI calculation has three components: expected loss, social vulnerability, and resilience capacity.  

## 1. Expected Loss
The EXH hazard expected loss (EL) includes consequences for the following receptors and marginal losses.
 * Shoreline / Loss of Ecosystem Services

#### 1a. Shoreline / Loss of Ecosystem Services
Annualized value of lost shoreline assuming 0.3 m coastal retreat (HMP 2019) and value of 1,977/acre/year for 
marine and estuary land.  Use CEHA dataset to identify length of eroding shoreline in each tract. Calculate 
cost over the next 100 years with discount rate equal to inflation.  

In [11]:
from MISC import params_1 as params
from MISC import utils_1 as utils
from MISC import plotting_1 as plotting
from CLASSES import class_EL_SV_CR_1 as URI
import matplotlib.pyplot as plt
import geopandas as gpd
import os
utils.set_home()
hazard_name = "CER"

#create ESL class and add loss
path_loss =   params.PATHNAMES.at['ESL_CER_ecosystem_loss', 'Value']
gdf_ESL_deaths = gpd.read_file(path_loss)
ESL = URI.ESL(hazard_name = hazard_name)
ESL.add_consequence("Ecosystem_Loss", "Shoreline", gdf_ESL_deaths)

#plot
plotting.plot_ESL_nb(hazard_name, ESL.consequence["Ecosystem_Loss"]["map_tract"], 'Loss USD - HAZUS')

DriverError: .\2_PROCESSED_INPUTS\4101_ESL_CER_LOSS_ECO.shp: No such file or directory

#### 1b. Expected Loss Total
The total expected losses are the sum of losses from each individual consequence.  

In [ ]:
#total all consequences
ESL.calc_ESL_total()

#save result
path_save = params.PATHNAMES.at['OUTPUTS_folder', 'Value'] + r'\ESL\ESL_{}_tract.shp'.format(hazard_name, hazard_name)
ESL.ESL_map.to_file(path_save)

#plot
plotting.plot_ESL_nb(hazard_name, ESL.ESL_map, 'Loss USD - Total')


## 2. Social Vulnerability Score
The Social Vulnerability is based on the CDC SoVI score.

In [ ]:
path_SOV = params.PATHNAMES.at['OUTPUTS_folder', 'Value'] + r'\\SOV\SOV_tract.shp'
gdf_SOV = gpd.read_file(path_SOV)
SOV = URI.SOV(hazard_name = hazard_name, map_tract = gdf_SOV)


#plot
plotting.plot_SOV_nb(SOV.SOV_map)

## 3. Resilience Capacity Score
The Resilience Capacity is the sum of four scores:
- Community capital
- Mitigation Landscape
- Response Capacity
- Recovery Resources


In [ ]:
path_RCA = params.PATHNAMES.at['OUTPUTS_folder', 'Value'] + r'\\RCA\RCA_{}_tract.shp'.format(hazard_name)
gdf_RCA = gpd.read_file(path_RCA)
RCA = URI.RCA(hazard_name=hazard_name, map_tract=gdf_RCA)
# #plot
# fig = plt.figure(figsize=(10, 5))
# ax = fig.add_subplot(111)
# RCA.RCA_map.plot(ax=ax, column = 'RCA', legend=True)
# plt.axis('off')
# plt.title('Resilience Capacity Score')

#
plotting.plot_RCA_nb('EXH', RCA.RCA_map)

## 4. Calculate URI Score
The URI is ESL * SOV / RCA.  The result is clustered into a score of 1 (low) to 5 (high).

In [ ]:
URI_HAZ = URI.URI(hazard_name=hazard_name, ESL=ESL, SOV=SOV, RCA=RCA)
URI_HAZ.calc_URI()

#save result
path_save = params.PATHNAMES.at['OUTPUTS_folder', 'Value'] + r'\URI\URI_{}_tract.shp'.format(hazard_name, hazard_name)
URI_HAZ.URI_map.to_file(path_save)


#plot
# fig = plt.figure(figsize=(10, 5))
# ax = fig.add_subplot(111)
# URI_EXH.URI_map.plot(ax=ax, column = 'URI', legend=True)
# plt.axis('off')
# plt.title('URI Score')
plotting.plot_URI_nb(hazard_name, URI_HAZ.URI_map)